This is a custom viewer for Hi-C data. I don't know why I did this when other, better viewers are available for free, but what the hell. Currently, it's quite janky and uses a fixed field of view at four zoom rates. It inter-operates with some confusing Python code. I should probably clean all this up...

Desired functionality:

(1) Show Hi-C data at multiple resolutions
(2) Use multiple colormaps
(3) Allow on-the-fly contrast adjustment
(4) Allow positions to be clicked and stored
(5) Show genomic data alongside Hi-C maps [this could be greatly improved]

Improvements:

-Make it so it doesn't demand mel chromosomes<br>
-Improve stability

In [10]:
'''
Set folders and files:
    folder = folder containing viewer data files
    save_folder = folder to save viewer snapshots to
    track_folder = folder containing genomic data tracks
'''
folder = '/Users/MStadler/Bioinformatics/Projects/HiC/data_2018/viewer_data/HiC-53_54_multi'
save_folder = '/Users/MStadler/Bioinformatics/Projects/HiC/data_2018/visual_inspections/HiC-53_54'
track_folder = '/Users/MStadler/Bioinformatics/Projects/HiC/data_2018/viewer_data/viewer_tracks/'

In [ ]:
import matplotlib as mpl

##mpl.use('TkAgg')

In [8]:
import matplotlib as mpl
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, RadioButtons, Cursor, TextBox, MultiCursor
import tkinter as tk
from tkinter import filedialog
import re
import ipywidgets as widgets
from IPython.display import display
import PyQt5
%matplotlib qt

#bug: if I start with inline then change it to qt it's fine...otherwise error if starting qt

In [5]:
# Function to save the list of clicked coordinates to a file 
def save_coord_list(filename_coords, coord_list):
   with open(filename_coords, 'w') as file_coords: 
        for i in range(0,len(coord_list)):
            chrom = coord_list[i][0]
            pos1 = str(int((coord_list[i][1] * 1000000) / 500) * 500)
            pos2 = str(int((coord_list[i][2] * 1000000) / 500) * 500)
            file_coords.write(chrom + ':' + pos1 + '-' + pos2 + '\t' + chrom + '\t' + pos1 + '\t' + pos2 + '\n')

In [6]:
''' 
Function to load Hi-C reduced matrix, flatten the top and bottom 1% of the data, takes log, 
scales to 1-1000 to account for negative numbers 
'''
def load_process_hic_matrix(infilename):
    x = np.genfromtxt(infilename, delimiter='\t', skip_header=1)
    x = x[:,1:] #take off first row which is generally bin names
    # Next two lines take care of 0's and nan's by setting to dummy value (typically 0.1)
    dummy_val = 0.1
    upper_trim = 99
    x[x == 0] = dummy_val
    x[np.isnan(x)] = dummy_val
    top = np.percentile(x, upper_trim)
    x[x > top] = top
    x = np.log(x)
    x = (x - np.min(x)) / (np.max(x) - np.min(x)) * 1000 # scales to 1-1000 betwixt min and max values.
    return(x)

In [11]:
'''
Main function, calls viewer
'''
#for 500 bp: 1000 bins 250,000 bp step

#######################################################################
# Core functions and classes, nuts and bolts of the viewer
#######################################################################

#Position class stores the information about the current state of the system (position, colormap, step size, etc.)
class Position:
    def __init__(self):
        Position.chr = "2R"
        Position.binsize = 500
        Position.L = 9000000
        Position.cmap = []
        Position.step_size = 100000
        Position.window_size = 200000
        Position.track_data = {}

#######################################################################
#linearly combine two colormaps—required a bit of work

def combine_colormaps(cm1, cm2):
    colors1 = cm1(np.linspace(0., 1, 128))
    colors2 = cm2(np.linspace(0., 1, 128))
    colors = np.vstack((colors1, colors2))
    return(mpl.colors.LinearSegmentedColormap.from_list('my_colormap', colors))

#######################################################################
#general function to update genome location when position is changed

def load_chr_pos():
    posL = pos.L
    posR = posL + pos.window_size
    new_file = os.path.join(folder, '_'.join([pos.chr, str(posL), str(posR)]) + '_' + str(pos.binsize) + '.txt')
    if (os.path.isfile(new_file) or os.path.isfile(new_file + '.gz')): 
        new_img = load_process_hic_matrix(new_file)
        img.set_data(new_img)
        L_Mb = float(posL) / 1e6
        R_Mb = float(posR) / 1e6
        img.set_extent([L_Mb, R_Mb, R_Mb, L_Mb])
        tloc.set_val(str(L_Mb))
        ax.set_title(pos.chr + ': ' + str(L_Mb) + '-' + str(R_Mb) + ' Mb', fontsize=14)
        img.set_cmap(pos.cmap)
        fig.canvas.draw_idle()
    else:
        ax.set_title('Position is out of bounds!', fontsize=14)
        
    #update WIG track data
    track_binsize = 500
    track_binL = int(posL / track_binsize)
    track_binR = int(track_binL + int(pos.window_size / track_binsize))
    ax_track.cla()
    ax_track.plot(pos.track_data[pos.chr][range(track_binR, track_binL, -1)], range(track_binL, track_binR))
    ax_track.set_ylim(track_binL, track_binR)

#######################################################################
#function for loading a new genomic dataset ('track')

def load_track_data(track):
    if (track == 'S5_DNase'): 
        track_file = 'S5_DNase_rep1_4col_dm6_binned500.txt'
    if (track == 'S5_RNAP2'):
        track_file = 'S5_polII_FDR25_4col_dm6_binned500.txt'
    if (track == 'BEAF-32 0-12h'):
        track_file = 'BEAF-32_0_12h_4col_dm6_binned500.txt'
    if (track == 'dCTCF 0-12h'):
        track_file = 'dCTCF_0_12h_4col_dm6_binned500.txt'
    if (track == 'CP190 0-12h'):
        track_file = 'CP190_0_12h_4col_dm6_binned500.txt'
    if (track == 'promoters'):
        track_file = 'dm6_promoters.txt'
    track_file_path = os.path.join(track_folder, track_file)
    max_chr_size = 4e7 #maximum allowable size of chromosomes for allocating arrays
    track_binsize = 500 #binsize of the track data
    pos.track_data = {}
    with open(track_file_path, 'r') as infile:
        for line in infile:
            items = line.split()
            (chr, posL, posR, val) = items
            chr = re.sub('chr', '', chr)
            if (chr not in pos.track_data):
                pos.track_data[chr] = np.zeros(int(max_chr_size / track_binsize))
            track_bin = int(int(posL) / track_binsize)
            pos.track_data[chr][track_bin] = float(val)    

#######################################################################
#Set up initial variables, containers, axes etc.
#######################################################################

pos = Position()
pos.cmap = combine_colormaps(plt.cm.Blues_r, plt.cm.YlOrBr)
axcolor="white"
fig, ax = plt.subplots(figsize=(10, 10))
fig.tight_layout()
plt.subplots_adjust(bottom=0.20, top=0.95)
folder_name_m = re.search(r'[^/]+$',folder) #matches everything after last slash
folder_name = folder_name_m.group()
fig.text(0.05, 0.05, folder_name, fontsize=20)
img = plt.imshow(np.zeros((100,100)), vmin=0, vmax=1000, cmap=pos.cmap, interpolation="none")
ax_track = plt.axes([0.8, 0.2, 0.1, 0.75],) #axes for WIG track data

#######################################################################
#Functions for the interactive features of the viewer
#######################################################################
#######################################################################
# Press any key to store the cursor position in coords

coords = []
def onkeypress(event):
    if(event.inaxes == ax):
        global coords
        coords.append((pos.chr, event.xdata, event.ydata))
        print((pos.chr, event.xdata, event.ydata))

cid = fig.canvas.mpl_connect('key_press_event', onkeypress)

# Click mouse to display position on track
def onclick(event):
    if(event.inaxes == ax):
        plt.sca(ax_track) #set to active axis
       # print(plt.ylim()[0])
       # print(plt.ylim()[1])
        plt.hlines(plt.ylim()[1] - (event.ydata * 1e6 / Position.binsize) + plt.ylim()[0] + 1, plt.xlim()[0]*0.5,plt.xlim()[1] * 0.5)
cid = fig.canvas.mpl_connect('button_press_event', onclick)

def onrelease(event):
    if(event.inaxes == ax):
        load_chr_pos()
cid = fig.canvas.mpl_connect('button_release_event', onrelease)

#######################################################################
#Radio buttons to change colormap

ax_colors = plt.axes([0.05, 0.3, 0.10, 0.15])
colors = RadioButtons(ax_colors, ('BlOr', 'Rd', 'RdGy'), active=0)

def update_colors(selection):
    if (selection == 'BlOr'):
        colormap = combine_colormaps(plt.cm.Blues_r, plt.cm.YlOrBr)
    elif (selection == 'Rd'):
        colormap = plt.cm.Reds
    elif (selection == 'RdGy'):
        colormap = plt.cm.RdGy_r
    pos.cmap = colormap
    img.set_cmap(pos.cmap)
    fig.canvas.draw_idle()   
colors.on_clicked(update_colors)

#######################################################################        
#Radio buttons to change chromosome

ax_chrom = plt.axes([0.05, 0.5, 0.05, 0.15])
chrom = RadioButtons(ax_chrom, ('2L', '2R', '3L', '3R', 'X'), active=0)

def update_chromosome(selection):
    pos.chr = selection
    load_chr_pos()
chrom.on_clicked(update_chromosome)

#######################################################################
#Radio buttons to change bin size (resolution)

ax_binsize = plt.axes([0.05, 0.8, 0.05, 0.15])
binsize = RadioButtons(ax_binsize, ('500', '1000', '2000', '4000'), active=0)

def update_binsize(selection):
    pos.binsize = int(selection)
    pos.step_size = int(selection) * 200
    pos.window_size = int(selection) * 400
    pos.L = int(pos.L / pos.step_size) * pos.step_size #finds nearest position that is a valid start position for this bin size
    load_chr_pos()
binsize.on_clicked(update_binsize)

#######################################################################
#Clickable button to save current viewer image as png into save_folder designated at top

ax_savefig = plt.axes([0.05, 0.7, 0.07, 0.05]) #left, bottom, width, height
savefig = Button(ax_savefig, 'Save', color="orange")

def execute_savefig(selection):
    file_path = os.path.join(save_folder, (pos.chr + '_' + str(pos.L) + '.png'))
    fig_saving, ax_saving = plt.subplots()
    plt.imshow(img.get_array(), vmin=scontrastlower.val, vmax=scontrastupper.val, cmap = pos.cmap, interpolation="none", extent = img.get_extent())
    mpl.pyplot.savefig(file_path)
    mpl.pyplot.close()
savefig.on_clicked(execute_savefig)

#######################################################################
#Editable textbox storing genomic position (displays and receives values)

ax_position_text = plt.axes([0.50, 0.03, 0.05, 0.05], facecolor=axcolor)
tloc = TextBox(ax_position_text, 'Mb')

def location_text_callback(text):
    pos_entered = int(float(text)* 1000000) #position in megabases
    if (pos_entered % pos.step_size == 0):
        pos.L = pos_entered
        load_chr_pos()
tloc.on_submit(location_text_callback)

#######################################################################
#contrast adjustment sliders

ax_contrast_lower = plt.axes([0.25, 0.1, 0.5, 0.02], facecolor=axcolor)
ax_contrast_upper = plt.axes([0.25, 0.13, 0.5, 0.02], facecolor=axcolor)
scontrastlower = Slider(ax_contrast_lower, 'Lower', 0, 1000, valinit=0, valstep=1, valfmt="%1.0f", fill=False, edgecolor="blue", linewidth=1)
scontrastupper = Slider(ax_contrast_upper, 'Upper', 0, 1000, valinit=1000, valstep=1, valfmt="%1.0f", fill=False, edgecolor="orange", linewidth=1)

def update_contrast(val):
    lower = int(scontrastlower.val)
    img.set_norm(mpl.colors.Normalize(vmin=scontrastlower.val, vmax=scontrastupper.val))
    fig.canvas.draw_idle()
scontrastlower.on_changed(update_contrast)
scontrastupper.on_changed(update_contrast)

#######################################################################
#left and right scroll buttons

ax_200kb_L = plt.axes([0.40, 0.03, 0.05, 0.05])
ax_200kb_R = plt.axes([0.58, 0.03, 0.05, 0.05])
ax_1Mb_L = plt.axes([0.30, 0.03, 0.05, 0.05])
ax_1Mb_R = plt.axes([0.68, 0.03, 0.05, 0.05])

b200L = Button(ax_200kb_L, '<', color="dodgerblue", hovercolor="darkorange")
b200R = Button(ax_200kb_R, '>', color="dodgerblue", hovercolor="darkorange")
b1MbL = Button(ax_1Mb_L, '<<', color="dodgerblue", hovercolor="darkorange")
b1MbR = Button(ax_1Mb_R, '>>', color="dodgerblue", hovercolor="darkorange")

def update_b200L(click):
    pos.L = int(pos.L - pos.step_size)
    load_chr_pos()
b200L.on_clicked(update_b200L)

def update_b200R(click):
    pos.L = int(pos.L + pos.step_size)
    load_chr_pos()
b200R.on_clicked(update_b200R)

def update_b1MbL(click):
    pos.L = int(pos.L - (pos.step_size * 5))
    load_chr_pos()
b1MbL.on_clicked(update_b1MbL)

def update_b1MbR(click):
    pos.L = int(pos.L + (pos.step_size * 5))
    load_chr_pos()
b1MbR.on_clicked(update_b1MbR)

#######################################################################
# Dropdown menu to display track data

load_track_data('S5_DNase')
load_chr_pos()

def track_selector_callback(change):
    if change['type'] == 'change' and change['name'] == 'value':
        load_track_data(track_selector.value)
        load_chr_pos()
    
track_selector = widgets.Dropdown(
    options=['S5_DNase', 'S5_RNAP2', 'BEAF-32 0-12h', 'dCTCF 0-12h', 'CP190 0-12h', 'promoters'],
    value='S5_DNase',
    description='Track:',
    disabled=False,
)
track_selector.observe(track_selector_callback)
display(track_selector)

#######################################################################
#Multicursor
#multi = MultiCursor(fig.canvas, (ax_track, ax), color='r', lw=1, horizOn=True, vertOn=True)

#Draws a crosshair cursor for visually aligning Hi-C features with genomic features
cursor = Cursor(ax, color="black")
m = Map()

def handle_interaction(**kwargs):
    if kwargs.get('type') == 'mousemove':
        pass

m.on_interaction(handle_interaction)


Dropdown(description='Track:', options=('S5_DNase', 'S5_RNAP2', 'BEAF-32 0-12h', 'dCTCF 0-12h', 'CP190 0-12h',…

NameError: name 'Map' is not defined

In [ ]:
save_coord_list('/Users/MStadler/Bioinformatics/Projects/HiC/data_2018/visual_inspections/20190619_HiC-59/20190702_1000bp_58_59_visualDiffs.txt',coords)

In [29]:
coords

[('2R', 9.077297035899186, 9.074326676907322),
 ('2R', 9.024455254025147, 9.032544802867383)]